In [5]:
import json
import websocket
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

In [8]:

url = 'wss://stream.binance.com:9443/ws'
symbols  = ["BTCUSDT", "AAVEUSDT", "STXUSDT", "ARBUSDT"]
df = pd.DataFrame()
project_id = 'binance-416514'
dataset_id = "data_stream"
table_id = "binance_streaming"

keypath ='/home/tin/credentials/binance-cred.json'
credentials = service_account.Credentials.from_service_account_file(
            keypath, 
            scopes= ["https://www.googleapis.com/auth/cloud-platform"],
            )

# credentials= keypath ,scopes= ["https://www.googleapis.com/auth/cloud-platform"]
client = bigquery.Client(credentials=credentials)




def on_message(ws,message):
    msg = json.loads(message)
    d = [(
        # msg['t'],
    msg['s'], msg['T'], msg['p'], msg['q'],msg['b'],msg['a'])]
    global df 
    df = pd.concat([df,pd.DataFrame.from_records(d)])

    # Define BigQuery table schema
    # schema = [
    #     bigquery.SchemaField('symbol', 'STRING'),
    #     bigquery.SchemaField('time', 'TIMESTAMP'),
    #     bigquery.SchemaField('price', 'FLOAT'),
    #     bigquery.SchemaField('qty', 'FLOAT'),
    # ]
    rows_to_insert = {
        # 'trade_id': msg['t'],
        'symbol': msg['s'],
        'time': pd.to_datetime(msg['E'], unit='ms').strftime('%Y-%m-%d %H:%M:%S'),
        'price': float(msg['p']),
        'qty': float(msg['q']),
        'buy_order_id' : msg['b'],
        'sell_order_id' : msg['a']
    }
    
    # Insert data into BigQuery table
    # rows_to_insert = [(msg['s'], msg['T'], msg['p'], msg['q'])]
    errors = client.insert_rows_json(
        f"{project_id}.{dataset_id}.{table_id}",
        [rows_to_insert],
        row_ids=[None]*len(rows_to_insert),
        skip_invalid_rows=True,
        ignore_unknown_values=True
    )

    if errors:
        print(f"Errors: {errors}")

def on_error(ws, error):
    print(error)

def on_close(ws,close_status_code, close_msg):
    
    df.columns = [
        'trade_id',
         'symbol','time','price','qty','buy_order_id','sell_order_id']
    df['time'] = pd.to_datetime(df['time'],unit = 'ms')
    df.set_index(df['time'],inplace = True)
    df.drop(columns='time',inplace = True)
    print(df)
    print('### closed ###')

def on_open(ws):
    print('open connection')
    for symbol in symbols:
        ws.send(json.dumps({
            "method": "SUBSCRIBE",
            "params": [
                f"{symbol.lower()}@trade"
            ],
            "id": 1
        }))

ws = websocket.WebSocketApp(url,
                            on_open =on_open,
                            on_message = on_message,
                            on_error = on_error,
                            on_close = on_close)

In [9]:
ws.run_forever()

open connection
's'
's'
's'
's'

Length mismatch: Expected axis has 6 elements, new values have 7 elements


True

In [15]:


url = 'wss://stream.binance.com:9443/ws'
symbols  = ["BTCUSDT", "AAVEUSDT", "STXUSDT", "ARBUSDT"]
df = pd.DataFrame()
project_id = 'binance-416514'
dataset_id = "data_stream"
table_id = "binance_streaming_candles"

keypath ='/home/tin/credentials/binance-cred.json'
credentials = service_account.Credentials.from_service_account_file(
            keypath, 
            scopes= ["https://www.googleapis.com/auth/cloud-platform"],
            )

# credentials= keypath ,scopes= ["https://www.googleapis.com/auth/cloud-platform"]
client = bigquery.Client(credentials=credentials)




def on_message(ws, message):
    msg = json.loads(message)
    kline = msg['k']  # Extract kline data

    rows_to_insert = {
        'Event_type': msg['e'],  # Event type
        'Event_time': pd.to_datetime(kline['t'], unit='ms').strftime('%Y-%m-%d %H:%M:%S'),  # Kline start time
        'Symbol': kline['s'],  # Symbol
        'Close_price': float(kline['c']),  # Close price
        'Open_price': float(kline['o']),  # Open price
        'High_price': float(kline['h']),  # High price
        'Low_price': float(kline['l']),  # Low price
        'Total_traded_base_asset_volume': float(kline['v']),  # Base asset volume
        'Total_traded_quote_asset_volume': float(kline['q']),  # Quote asset volume
        # Add more fields as needed
    }

    print(rows_to_insert)  # Just for testing

    # Insert data into BigQuery table
    errors = client.insert_rows_json(
        f"{project_id}.{dataset_id}.{table_id}",
        [rows_to_insert],
        row_ids=[None]*len(rows_to_insert),
        skip_invalid_rows=True,
        ignore_unknown_values=True
    )

    if errors:
        print(f"Errors: {errors}")

    

    # Insert data into BigQuery table
    # rows_to_insert = [(msg['s'], msg['T'], msg['p'], msg['q'])]
    errors = client.insert_rows_json(
        f"{project_id}.{dataset_id}.{table_id}",
        [rows_to_insert],
        row_ids=[None]*len(rows_to_insert),
        skip_invalid_rows=True,
        ignore_unknown_values=True
    )

    if errors:
        print(f"Errors: {errors}")

def on_error(ws, error):
    print(error)

def on_close(ws,close_status_code, close_msg):
    
    df.columns = [
        "Event_type",
        "Event_time",
        "Symbol",
        "Close_price",
        "Open_price",
        "High_price",
        "Low_price",
        "Total_traded_base_asset_volume",
        "Total_traded_quote_asset_volume"]
    # df['time'] = pd.to_datetime(df['time'],unit = 'ms')
    # df.set_index(df['time'],inplace = True)
    # df.drop(columns='time',inplace = True)
    print(df)
    print('### closed ###')

def on_open(ws):
    print('open connection')
    for symbol in symbols:
        ws.send(json.dumps({
            "method": "SUBSCRIBE",
            "params": [
                f"{symbol.lower()}@kline_1M"
            ],
            "id": 1
        }))

ws2 = websocket.WebSocketApp(url,
                            on_open =on_open,
                            on_message = on_message,
                            on_error = on_error,
                            on_close = on_close)

In [16]:
ws2.run_forever()

open connection
'k'
'k'
'k'
'k'
{'Event_type': 'kline', 'Event_time': '2024-03-01 00:00:00', 'Symbol': 'BTCUSDT', 'Close_price': 71088.32, 'Open_price': 61130.99, 'High_price': 73000.0, 'Low_price': 59005.0, 'Total_traded_base_asset_volume': 722569.831449, 'Total_traded_quote_asset_volume': 48254009204.18434}
{'Event_type': 'kline', 'Event_time': '2024-03-01 00:00:00', 'Symbol': 'ARBUSDT', 'Close_price': 2.0168, 'Open_price': 1.9447, 'High_price': 2.2783, 'Low_price': 1.765, 'Total_traded_base_asset_volume': 1382786295.9, 'Total_traded_quote_asset_volume': 2848300769.29424}
{'Event_type': 'kline', 'Event_time': '2024-03-01 00:00:00', 'Symbol': 'BTCUSDT', 'Close_price': 71088.31, 'Open_price': 61130.99, 'High_price': 73000.0, 'Low_price': 59005.0, 'Total_traded_base_asset_volume': 722570.385639, 'Total_traded_quote_asset_volume': 48254048600.61514}
{'Event_type': 'kline', 'Event_time': '2024-03-01 00:00:00', 'Symbol': 'ARBUSDT', 'Close_price': 2.0166, 'Open_price': 1.9447, 'High_price':

True